In [1]:
import argparse
import math
import os
import time

import imageio
import numpy as np
import torch
import torch.nn.functional as F
import tqdm
from datasets.dnerf_synthetic import SubjectLoader
from radiance_fields.mlp import DNeRFRadianceField
from utils import render_image, set_random_seed

from nerfacc import ContractionType, OccupancyGrid

/home/loyot/anaconda3/envs/nerfacc/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = "cuda:0"
set_random_seed(42)

parser = argparse.ArgumentParser()
parser.add_argument(
    "--train_split",
    type=str,
    default="train",
    choices=["train"],
    help="which train split to use",
)
parser.add_argument(
    "--scene",
    type=str,
    default="lego",
    choices=[
        # dnerf
        "bouncingballs",
        "hellwarrior",
        "hook",
        "jumpingjacks",
        "lego",
        "mutant",
        "standup",
        "trex",
    ],
    help="which scene to use",
)
parser.add_argument(
    "--aabb",
    type=lambda s: [float(item) for item in s.split(",")],
    default="-1.5,-1.5,-1.5,1.5,1.5,1.5",
    help="delimited list input",
)
parser.add_argument(
    "--test_chunk_size",
    type=int,
    default=8192,
)
parser.add_argument("--cone_angle", type=float, default=0.0)
args = parser.parse_args(["--train_split", "train", "--scene", "lego"])

In [3]:
render_n_samples = 1024

# setup the scene bounding box.
contraction_type = ContractionType.AABB
scene_aabb = torch.tensor(args.aabb, dtype=torch.float32, device=device)
near_plane = None
far_plane = None
render_step_size = (
    (scene_aabb[3:] - scene_aabb[:3]).max()
    * math.sqrt(3)
    / render_n_samples
).item()

# setup the radiance field we want to train.
max_steps = 30000
grad_scaler = torch.cuda.amp.GradScaler(1)
radiance_field = DNeRFRadianceField().to(device)
optimizer = torch.optim.Adam(radiance_field.parameters(), lr=5e-4)
scheduler = torch.optim.lr_scheduler.MultiStepLR(
    optimizer,
    milestones=[
        max_steps // 2,
        max_steps * 3 // 4,
        max_steps * 5 // 6,
        max_steps * 9 // 10,
    ],
    gamma=0.33,
)
# setup the dataset
data_root_fp = "/home/loyot/workspace/Datasets/NeRF/dynamic_data/"
target_sample_batch_size = 1 << 16
grid_resolution = 128

train_dataset = SubjectLoader(
    subject_id=args.scene,
    root_fp=data_root_fp,
    split=args.train_split,
    num_rays=target_sample_batch_size // render_n_samples,
).to(device)

test_dataset = SubjectLoader(
    subject_id=args.scene,
    root_fp=data_root_fp,
    split="test",
    num_rays=None,
).to(device)

In [4]:
test_dataset.timestamps

tensor([[0.0000],
        [0.0505],
        [0.1010],
        [0.1515],
        [0.2020],
        [0.2626],
        [0.3131],
        [0.3636],
        [0.4141],
        [0.4646],
        [0.5253],
        [0.5758],
        [0.6263],
        [0.6768],
        [0.7273],
        [0.7879],
        [0.8384],
        [0.8889],
        [0.9394],
        [1.0000]], device='cuda:0')